In [1]:
import cv2
import numpy as np

In [2]:
img = cv2.imread('rebuild2.tif')
img_b,g,r = cv2.split(img)
cv2.imshow('origin(a)',img_b)

# 重建开操作
def rebuild(img, kernel, origin):
#     kernel = np.ones((3,3), np.uint8)
    rebuild_fore = cv2.dilate(img, kernel,iterations=1)
    # 和原图与操作
    rebuild_fore = np.minimum(rebuild_fore, origin)
    rebuild_next = cv2.dilate(rebuild_fore, kernel, iterations=1)
    rebuild_next = np.minimum(rebuild_next, origin)

    while not (rebuild_fore == rebuild_next).all():
        rebuild_fore = rebuild_next
        rebuild_next = cv2.dilate(rebuild_next, kernel, iterations=1)
        rebuild_next = np.minimum(rebuild_next, origin)

    return rebuild_next

    
#重建开操作(1,71)
kernel = np.ones((1,71), np.uint8)
erode = cv2.erode(img_b,kernel,iterations=1)
# cv2.imshow('erode', erode)
rebuild_erode = rebuild(erode, kernel, img_b)
cv2.imshow('rebuild(1,71)(b)', rebuild_erode)

#开操作
opening = cv2.morphologyEx(img_b, cv2.MORPH_OPEN, kernel)
cv2.imshow('open(c)', opening)

#重建顶帽操作
rebuild_T_hot = img_b - rebuild_erode
cv2.imshow('rebuild_T_hot(d)',rebuild_T_hot)

#顶帽操作
T_hot = img_b - opening
cv2.imshow('T_hot(e)', T_hot)

# f
kernel = np.ones((1,11), np.uint8)
erode_d = cv2.erode(rebuild_T_hot,kernel,iterations=1)
kernel = np.ones((3,3), np.uint8)
rebuild_d = rebuild(erode_d, kernel, rebuild_T_hot)
cv2.imshow('rebuild_d(f)', rebuild_d)

# g
kernel = np.ones((1,21), np.uint8)
dilate_f = cv2.dilate(rebuild_d, kernel, iterations=1)
cv2.imshow('dilate_f(g)', dilate_f)

# h
img_h = np.minimum(rebuild_T_hot, dilate_f)
cv2.imshow('minimum(h)',img_h)

# i
kernel = np.ones((3,3), np.uint8)
img_i = rebuild(img_h, kernel, rebuild_T_hot)
cv2.imshow('result(i)', img_i)


cv2.waitKey(0)
cv2.destroyAllWindows()